In [143]:
import numpy as np

In [151]:
# matrix to decompose (X = A @ B.T)
N = 5

X_tmp = np.random.rand(N, N) * 1000
X = X_tmp @ X_tmp.T
print(f'Rank(X) = {np.linalg.matrix_rank(X)}')

# init. addiyional matrices
A = np.linalg.cholesky(X)
B = A.copy()
assert np.allclose(X, A @ B.T)

K = X.T @ A
G = A.T @ A

U = np.random.rand(*X.shape)

Rank(X) = 5


In [152]:
def ADMM(B, U, K, G, *, rank, epsilon):
    
    ro = np.trace(G) / rank
    L = np.linalg.cholesky(G + ro * np.eye(G.shape[0]))
    
    r = np.inf
    s = np.inf
    
    while (r > epsilon) or (s > epsilon):
        B_ = np.linalg.inv(L).T @ np.linalg.inv(L) @ (K + ro * (B + U)).T
        B0 = B.copy()
        B = (B_.T - U).astype('int32')
        U += B - B_.T
        r = np.linalg.norm(B - B_.T, ord='fro')**2 / np.linalg.norm(B, ord='fro')**2
        s = np.linalg.norm(B - B0, ord='fro')**2 / np.linalg.norm(U, ord='fro')**2
    
    return B, U

In [157]:
epsilon = 1
B_new, U = ADMM(B, U, K, G, rank=3, epsilon=epsilon)

In [158]:
assert np.allclose(A @ B_new.T, X, atol=epsilon, rtol=epsilon)

In [159]:
B_new

array([[1309,    0,    0,    0,    0],
       [1231,  507,    0,    0,    0],
       [1023,  732,  727,    0,    0],
       [1084,   -9,  -46,  482,    0],
       [1033,  144,  109,  955,  209]], dtype=int32)

In [162]:
B.astype('int32')

array([[1308,    0,    0,    0,    0],
       [1231,  506,    0,    0,    0],
       [1023,  732,  727,    0,    0],
       [1084,   -9,  -45,  481,    0],
       [1032,  143,  109,  955,  209]], dtype=int32)